In [1]:
import pandas as pd
import numpy as np
#import gzip
#import json
import sys
from langdetect import detect, LangDetectException
import nltk
#import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from nltk.sentiment import SentimentIntensityAnalyzer
#from sklearn.decomposition import PCA
#from scipy.stats import zscore
#from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import joinLanguage
#from myFunctions import jsonGzipToDataframe2, toDommyColumns, joinLanguage

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Descargamos el paquete de palabras que utilizara el analizador se sentimientos

In [2]:
nltk.download('vader_lexicon') # ¡SE DEBE EJECUTAR SOLO UNA VEZ!

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/rafael/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Cargamos dentro de dataframes Pandas los CSVs comprimido 

In [3]:
df = pd.read_csv('../datasource/user_reviews_chewed.csv.gz')

In [4]:
dfSteam = pd.read_csv('../datasource/steam_games_chewed.csv.gz')

Nos quedaremos con las columas que necesitaremos agregar a df

In [5]:
dfSteam = dfSteam[['id', 'developer']]

Quitamos los 'developer' que tengan valor NaN 

In [6]:
dfSteam = dfSteam.dropna(subset = ['developer']).reset_index(drop = True)

In [7]:
dfSteam

,id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,772540,Trickjump Games Ltd
...,...,...
28823,745400,Bidoniera Games
28824,773640,"Nikita ""Ghost_RUS"""
28825,733530,Sacada
28826,610660,Laush Dmitriy Sergeevich


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263226 entries, 0 to 263225
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      263226 non-null  object 
 1   item_id      66616 non-null   float64
 2   posted       66616 non-null   object 
 3   last_edited  6140 non-null    object 
 4   recommend    66616 non-null   object 
 5   funny        8151 non-null    object 
 6   helpful      66616 non-null   object 
 7   review       66586 non-null   object 
dtypes: float64(1), object(7)
memory usage: 16.1+ MB


Quitamos los reviews que tengan valor NaN 

In [9]:
df = df.dropna(subset = ['review']).reset_index(drop = True)

Ajustamos rapidamente el tipo de dato de item_id

In [10]:
df['item_id'] = df['item_id'].astype('int64')  # Convierte a entero de 64 bits

#### Save 💾

En este punto el dataframe df esta listo para ser la fuente de datos de la función 'userdata()' que respondera desde la<br>API. Siendo asi, tomaremos una instantanea del estado actual de los datos y los persistiremos en un CSV. 

In [11]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_parquet('../datasource/render/user_reviews_4Api.parquet')
#df.to_csv('../datasource/render/user_reviews_4Api.csv.gz', compression='gzip', index = False)

Y nos quedamos con las columnas que se van a utilizar en el analisis de sentimiento

In [12]:
df = df[['item_id', 'posted', 'recommend', 'review']]

In [13]:
df.head()

,item_id,posted,recommend,review
0,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


In [14]:
df['recommend'].value_counts()

recommend
True     59757
False     6829
Name: count, dtype: int64

Agreguemos a df la columna 'developer' que esta en dfSteam

In [15]:
# Let's add the 'gamesGenre' columns to 'dfUserItems'. Now we have the years of the games
# next to the items!
dfUnion = df.merge(dfSteam, left_on = 'item_id', right_on = 'id', how = 'inner')

Obtengamos el año del comentario del contenido de la columna 'posted' y guardemoslo en una nueva columna 'year'

In [16]:
# Verifiquemos el formato de fecha valido de la columna 'posted'
# Usando la exresion regular 'Posted |,|\.' podemos verificar la forma en la que esta escrita la 
# fecha. Estamos buscando un texto como por ejemplo:
#              Posted xxx, yyyy.
# En donde yyyy sera el año
dfUnion['posted'] = pd.to_datetime(
    dfUnion['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex = True), errors = 'coerce')

# Crea la columna 'year' a partir de 'posted'
dfUnion['year'] = dfUnion['posted'].dt.year.astype('Int64')

# La columna posted ya no hace falta y aprovecho para quitar 'id'
dfUnion.drop(['posted', 'id'], axis = 1, inplace = True)

Quitamos los años que tengan valor NaN 

In [17]:
dfUnion = dfUnion.dropna(subset = ['year']).reset_index(drop = True)

In [18]:
dfUnion

,item_id,recommend,review,developer,year
0,1250,True,Simple yet with great replayability. In my opi...,Tripwire Interactive,2011
1,1250,True,"Amazing, Non-stop action of blowing stuff to b...",Tripwire Interactive,2015
2,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",Tripwire Interactive,2013
3,1250,True,Jogo ♥♥♥♥.,Tripwire Interactive,2015
4,1250,True,cara nas imagens esse jogo da pouco de medo ma...,Tripwire Interactive,2014
...,...,...,...,...,...
48889,367780,True,The game is a good game. I might understand th...,"Soloweb Studios,Ravens Eye Studio",2015
48890,367780,True,I can understand why Aero's Quest brings out s...,"Soloweb Studios,Ravens Eye Studio",2015
48891,305920,False,i really didn't like it. i'm sorry. slow and b...,ShaunJS,2015
48892,306040,True,"Simple and fun, great art work.","Antanas Marcelionis,Renė Petrulienė",2015


#### Idioma 🇬🇧

In [19]:
dfUnionLanguage = dfUnion.copy()

In [20]:
#print('dfUnionLanguage:', id(dfUnionLanguage))
print('dfUnion', id(dfUnion))
print('dfUnionLanguage:', id(dfUnionLanguage))

dfUnion 139903710210960
dfUnionLanguage: 139903622484736


Agreguemos la columna 'language' que corresponde al idioma del comentario del usuario

In [21]:
dfUnionLanguage = joinLanguage(dfUnionLanguage, 'review')


De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [22]:
print('Reseñas nulas:',dfUnionLanguage['language'].isnull().sum())
dfUnionLanguage['language'].fillna(1, inplace=True)
print('Reseñas nulas despues de la conversion:',dfUnionLanguage['language'].isnull().sum())

Reseñas nulas: 391
Reseñas nulas despues de la conversion: 0


¿Cual es el idioma que mas se repite?

In [23]:
dfUnionLanguage['language'].value_counts().head()

language
en    39387
pt     1539
es      913
de      831
so      707
Name: count, dtype: int64

Nos quedaremos con 'en'

In [24]:
# Borramos los comentarios que no estan en ingles y los que tienen valor NaN
dfUnionLanguage = dfUnionLanguage.loc[
      (dfUnionLanguage['language'] == 'en')
      #(dfUnionLanguage['language'].notna()) & (dfUnionLanguage['language'] == 'en')
    ].reset_index(drop = True)
# Todo quedo en ingles asi que es redundante la columna 'language'
dfUnionLanguage.drop('language', axis = 1, inplace = True)

In [25]:
dfUnionLanguage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39387 entries, 0 to 39386
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    39387 non-null  int64 
 1   recommend  39387 non-null  object
 2   review     39387 non-null  object
 3   developer  39387 non-null  object
 4   year       39387 non-null  Int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 1.5+ MB


Proceso de limpieza de la columna 'review'

In [26]:
# Cambiemos todo a minusculas para evitar ambiguedades en las palabras.
dfUnionLanguage.loc[:, 'review'] = dfUnionLanguage['review'].str.lower()

# Fuera los caracteres especiales y signos de puntuacion.
dfUnionLanguage['review'] = dfUnionLanguage['review'].replace('[^A-Za-z0-9\s]+', '', regex = True)
dfUnionLanguage.loc[:, 'review'] = dfUnionLanguage['review'].str.replace('[^\w\s]', '', regex = True)

#### Analisis de sentimiento 🫶

In [27]:
def sentimentAnalysis(df):
    analyzer = SentimentIntensityAnalyzer()

    # Calcular las puntuaciones de sentimiento a cada texto en la columna 'review'
    # Creamos la columna 'compoundScore' para guardad el calculo
    df['compoundScore'] = df['review'].apply(
        lambda review: analyzer.polarity_scores(review)['compound'])
    # Reflejamos en la nueva columna 'sentimentAnalysis' la escala '0' si es malo,
    # '1' si es neutral y '2' si es positivo
    df['sentimentAnalysis'] = df['compoundScore'].apply(
        lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    scoreCounts = df['sentimentAnalysis'].value_counts()

    # Conteo de reviews en blanco
    blankReviewsCount = df['review'].isnull().sum()

    # Total de reviews
    totalReviews = len(df)

    dataset = {
      # Conteo de reviews por score
      'scoreCounts': scoreCounts,

      # Conteo de reviews en blanco
      'blankReviewsCount': blankReviewsCount,

      # Calcular porcentajes
      'scorePercentages': (scoreCounts / totalReviews * 100).round(2),
      'blankReviewsPercentage': (blankReviewsCount / totalReviews * 100).round(2)
    }
     
    # Ya no necesitaremos las columnas 'review' y 'compoundScore'
    df.drop(['review','compoundScore'], axis = 1, inplace = True)  

    #return df, dataset

In [28]:
# Ejecutamos el analisis de sentimiento y lo incorporamos como una nueva columna al dataframe
sentimentAnalysis(dfUnionLanguage)
#dfUserReviews, d = sentimentAnalysis(dfUnionLanguage)

In [29]:
dfUnionLanguage

,item_id,recommend,developer,year,sentimentAnalysis
0,1250,True,Tripwire Interactive,2011,2
1,1250,True,Tripwire Interactive,2015,2
2,1250,True,Tripwire Interactive,2013,0
3,1250,True,Tripwire Interactive,2010,2
4,1250,True,Tripwire Interactive,2015,0
...,...,...,...,...,...
39382,367780,True,"Soloweb Studios,Ravens Eye Studio",2015,2
39383,367780,True,"Soloweb Studios,Ravens Eye Studio",2015,2
39384,305920,False,ShaunJS,2015,0
39385,306040,True,"Antanas Marcelionis,Renė Petrulienė",2015,2


In [30]:
dfUnionLanguage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39387 entries, 0 to 39386
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   item_id            39387 non-null  int64 
 1   recommend          39387 non-null  object
 2   developer          39387 non-null  object
 3   year               39387 non-null  Int64 
 4   sentimentAnalysis  39387 non-null  int64 
dtypes: Int64(1), int64(2), object(2)
memory usage: 1.5+ MB


#### Save 💾

En este punto el dataframe df esta listo para ser la fuente de datos de las funciones 'best_developer_year' y<br>'developer_reviews_analysis()' que responderan desde la API. Siendo asi, tomaremos una instantanea del estado actual<br>de los datos y los persistiremos en un CSV. 

In [31]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_parquet('../datasource/render/user_reviews_2Api.parquet')
#dfUnionLanguage.to_csv('../datasource/render/user_reviews_2Api.csv.gz', compression='gzip', index = False)

#### LA CONSULTA PARA LA API

In [32]:
def best_developer_year(year: int) -> pd.DataFrame:
  """
  Devuelve el top 3 de desarrolladores con juegos mas recomendados por usuarios para el año dado.
  Params:
    year (int): El año a considerar.
  Returns:
    pd.DataFrame: Un DataFrame con las columnas 'developer' y 'recomendaciones', ordenado por
    'recomendaciones' de forma descendente y limitado a las 3 primeras filas.
  """
  if not isinstance(year, int):
    return print(f'{year} fuera de rango')
  # Filtrar los juegos recomendados para el año dado
  juegosRecomendados = dfUnionLanguage[
      (dfUnionLanguage['year'] == year) &
      (dfUnionLanguage['recommend'] == True) &
      (dfUnionLanguage['sentimentAnalysis'] == 2)
  ]  
  
  # Contar las recomendaciones por desarrollador
  recomendacionesXDesarrollador = (
      juegosRecomendados.groupby('developer')['item_id'].nunique()
  )  
  
  # Convertir a DataFrame y ordenar por recomendaciones de forma descendente
  topDesarrolladores = recomendacionesXDesarrollador.to_frame(name = 'recomendaciones').reset_index()
  topDesarrolladores = topDesarrolladores.sort_values(by = 'recomendaciones', ascending = False)  

  topDesarrolladoresLista = []
  for i in range(0, 3):
      puesto = f"Puesto {i + 1}"
      try:
        desarrollador = topDesarrolladores.loc[i, 'developer']
      except Exception as e:
         return print(f'{year} fuera de rango\n{e}')
      topDesarrolladoresLista.append({puesto: desarrollador})
  return topDesarrolladoresLista

In [33]:
best_developer_year(2010)

[{'Puesto 1': '1C Company'},
 {'Puesto 2': '1C: Maddox Games'},
 {'Puesto 3': 'BioWare'}]

In [34]:
def developer_reviews_analysis(developer: str) -> dict:
    """Devuelve un diccionario con el nombre del desarrollador como llave y una lista con la
    cantidad de reseñas negativas y positivas del desarrollador.
    Params:
        developer (str): El nombre del desarrollador.
    Returns:
        dict: Un diccionario con la siguiente estructura: {'nombre_desarrollador': ['Negative = X', 'Positive = Y']}
    """
    # Filtrar las reseñas del desarrollador especificado
    developerReviews = dfUnionLanguage[dfUnionLanguage['developer'] == developer]

    # Contar las reseñas positivas y negativas
    negativeCount = developerReviews[developerReviews['sentimentAnalysis'] == 0].shape[0]
    positiveCount = developerReviews[developerReviews['sentimentAnalysis'] == 2].shape[0]

    # Crear el diccionario de resultados
    resultado = {
        developer: [f"Negative = {negativeCount}", f"Positive = {positiveCount}"]
    }

    return resultado


In [35]:
developer_reviews_analysis('Tripwire Interactive')

{'Tripwire Interactive': ['Negative = 83', 'Positive = 244']}